### Libraries

In [2]:
import math

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import scipy as sp

from scipy.optimize import fsolve, root_scalar

from scipy.integrate import quad, dblquad, tplquad, nquad



# Root Finding

Newtons Method: $$ x_{n+1} = x_n - \frac {f(x_n)}{f'(x_n)}$$

with a for loop

In [2]:
r=2

f = lambda x: 1-x-np.exp(-r*x)
fp = lambda x: -1+r*np.exp(-r*x)

tol = 1e-08
root = 0.4
for i in range(100000):
    root = root-f(root)/fp(root)  
    
# print(f"the root is {root}")

# Difference Equations

In [3]:
def logistic(x, r=3.9):
    return r*x*(1-x)

# logistic map for a few values of r

rs = [2.5, 3, 3.5, 3.9, 4.0, 4.1]
# rs = np.linspace(2.5, 4.0, 6)

# plt.figure(figsize=(12, 8))

for r in rs:
    xs = [0.1]

    for i in range(60):
        xs.append(logistic(xs[-1], r))
        
    # plt.subplot(2, 3, rs.index(r)+1)
    # plt.plot(xs, '--x')
    # plt.title(f"Logistic map with r={r}")

In [4]:
# bifurcation diagram for logistic map

rs = np.linspace(2, 4, 1000)

# plt.figure(figsize=(12, 8))

for r in rs:
    xs = [0.1]
    for i in range(1000):
        xs.append(logistic(xs[-1], r=r))

    # plt.plot([r]*len(xs[-20:]), xs[-20:], '.', markersize=1)

In [5]:
# finding 10-periodic solutions of the logistic map
from scipy.optimize import fsolve
from functools import reduce 

period = 10

def composite(f, n):
    # no compositions, return identity
    if n == 0:
        return lambda x: x
    
    # apply composition recursively over [f, f, ..., f] (n times)
    return reduce(lambda f, g: lambda x: f(g(x)), [f]*n)


x = fsolve(lambda x: composite(logistic, period)(x)-x, 0.2)

# for i in range(period+1):
#     print(composite(logistic, i)(x))

# Numerical Differentiation

In [6]:
def fwd_diff(f, x, h=1e-8):
    return (f(x+h) - f(x)) / h

def ct_diff(f, x, h=1e-8):
    return (f(x+h) - f(x-h)) / (2*h)

# Interpolation

In [4]:
N = 20

xs = np.linspace(0, 0.6, N)
ys = np.sin(2*np.pi*xs) + 0.2*np.random.randn(N)

# the matrix containing the nth degree polynomials as rows evaluated at the xs
vandermond = lambda xs,n: np.matrix([[x**i for i in range(n)] for x in xs])
V = vandermond(xs, len(xs))

# solve for the coefficients of the polynomial
c = np.linalg.solve(V, ys)

# evaluate the polynomial at the xs
def poly(c,x):
    if x is np.ndarray:
        return np.array([sum([c[i]*x**i for i in range(len(c))]) for x in x])
    return sum([c[i]*x**i for i in range(len(c))])

# plot the polynomial
x_range = np.linspace(0, 0.6, 1000)
y_range = poly(c, x_range)

# plt.plot(x_range, y_range)
# plt.plot(xs, ys, '.')
# plt.ylim(-1.5, 1.5)

# Regression

In [ ]:
def line(c,x):
    if x is np.ndarray:
        return np.array([c[0]*x + c[1] for x in x])
    return c[0]*x + c[1]

def llsq(pts):
    '''
        Returns the coefficients of the linear least squares fit of the points
    '''
    if type(pts) != np.ndarray:
        # could further handle if shape is wrong. if not right shape, transpose it.
        print('pts is not an array')
        pts = np.array(pts).T
        pts = np.array([pts[:,0], np.ones(len(pts))]).T
        ys = pts[:,1]
    else:
        # could further handle if shape is wrong. if not right shape, transpose it.
        ys = pts[:,1]
        pts = np.array([pts[:,0], np.ones(len(pts))]).T
    
    c = np.linalg.solve(pts.T@pts, pts.T@ys)
    plt.title('Linear Least Squares Demo')
    plt.plot(pts[:,0], line(c, pts[:,0]), label=f'slope = {c[0]:0.0f}, intercept = {c[1]:0.1f}')
    plt.plot(pts[:,0], ys, '.', label='data')
    plt.grid()
    plt.legend()
    return c

# PCA

In [ ]:
def pax(X):
    '''
        Return eigenvalues and eigenvectors of the covariance matrix
    '''
    cov = X.T@X
    vals, e = np.linalg.eig(cov)

    plt.plot(X[:, 0], X[:, 1], '.')
    plt.plot([0, e[0, 0]], [0, e[1, 0]], 'r')
    plt.plot([0, e[0, 1]], [0, e[1, 1]], 'g')
    plt.axis('equal')
    return vals, e

# Random

In [20]:
def run_brownian(ts, sigma, B0):
    bs = [B0]
    diff = ts[1] - ts[0]
    for t in ts:
        bs.append(bs[-1] + sigma * np.random.normal(0, 1) * np.sqrt(diff))
    return np.array(bs)

# exponential of brownian motion
S0 = 10
ts = np.linspace(1, 10, 1000)
S = np.exp(run_brownian(ts, 0.01, 0)) * S0

# plt.plot(S)

equal
equal
equal
equal
equal
equal
equal


In [ ]:
# Binomial process
from numpy.random import binomial as B

for i in range(100):
    if sum(B(1, 0.5, 100)) == B(100, 0.5, 1):
        print('equal')

In [7]:
# 2d version

sigma = 1
B0 = [0, 0]
h = 0.1

X = [0,0]
Xs = [X]

for _ in range(1000):
    
    theta = np.random.random()*2*np.pi
    r = np.random.normal(0, np.sqrt(h)*sigma)

    X[0] = X[0] + r*np.cos(theta)
    X[1] = X[1] + r*np.sin(theta)

    Xs.append((X[0], X[1]))    

# plt.plot(*np.array(Xs).T)

# Numerical Integration

In [9]:
def rie(N, a, b, mesh=10):
    xs = np.linspace(a,b,mesh)
    tally = 0
    for i, x in enumerate(xs):
        if i==0 or i==len(xs)-1:
            continue
        dx = xs[i+1] - xs[i]
        yi = N(xs[i])
        tally += yi*dx
        plt.plot([x, x+dx], [yi, yi], 'r')
        for i,l in enumerate(np.linspace(x,x+dx, 4)):
            plt.plot([l, l], [0, yi], 'r')
    return tally

# rie(lambda x: np.sin(x), 0, np.pi, mesh=100)

In [14]:
from scipy.integrate import quad, dblquad, tplquad, nquad

# Common SymPy

In [12]:
from sympy import *

##################################################
# Spherical coordinates
rho, theta, phi = symbols('rho theta phi')

x = rho * sin(phi) * cos(theta)
y = rho * sin(phi) * sin(theta)
z = rho * cos(phi)

D = Matrix([x, y, z]).jacobian([rho, phi, theta])
##################################################

##################################################
# Solve
x, y = symbols('x y', real=True)
level_curve = Eq(y**2-y+x**2, 0.3)
funcs = []
for sol in solve(level_curve, y):
    funcs.append(sol)
# for func in funcs:
    # plot(func, (x,-10,10))
r1, r2 = solve(funcs[0]-funcs[1], x) # intersection points
float(integrate(funcs[1]-funcs[0], (x, r1, r2)))
##################################################


1.7278759594743864

# fsolve examples

In [15]:
from scipy.optimize import fsolve